# KML Writer - Keyhole Markup Language

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [GPS Wizard](https://logiqx.github.io/gps-wizard/) and is distributed under the terms of the GNU General Public License.

GPS Wizard is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

GPS Wizard is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with GPS Wizard. If not, see <https://www.gnu.org/licenses/>.

In [1]:
import os
import sys
import time

from lxml import etree
from datetime import datetime

import unittest

from base_writer import BaseWriter

## Main Class

In [2]:
class KmlWriter(BaseWriter):
    '''KML file - Keyhole Markup Language'''

    def __init__(self, filename=None, tracks=None):
        '''Basic init just records the filename'''

        super().__init__(filename, tracks=tracks)


    def prepare(self, tracks, name='GPS Wizard'):
        '''Prepare KML prior to being saved'''

        # XML document
        xsi = 'http://www.w3.org/2001/XMLSchema-instance'
        kml = etree.Element(
            'kml', 
             {etree.QName(xsi, 'schemaLocation'): 'http://www.opengis.net/kml/2.2 http://schemas.opengis.net/kml/2.2.0/ogckml22.xsd'},
             nsmap={None: 'http://www.opengis.net/kml/2.2', 'xsi': xsi})

        # Document
        document = etree.SubElement(kml, 'Document')
        documentName = etree.SubElement(document, 'name')
        documentName.text = name
        documentOpen = etree.SubElement(document, 'open')
        documentOpen.text = '1'

        # Style
        style = etree.SubElement(document, 'Style')
        style.attrib['id'] = 'wizard'

        # LineStyle
        linestyle = etree.SubElement(style, 'LineStyle')
        linestyleColor = etree.SubElement(linestyle, 'color')
        linestyleColor.text = 'a00000ff'
        linestyleWidth = etree.SubElement(linestyle, 'width')
        linestyleWidth.text = '2'       

        # Each track will be a new placemark
        for track in tracks:
            # Formatting of latitude and longitude
            formats = self.getFormats(track)
            coordinateFormat = '{},{}'.format(formats['lat'], formats['lon'])

            # Placemark
            placemark = etree.SubElement(document, 'Placemark')
            placemarkName = etree.SubElement(placemark, 'name')
            if track.name:
                placemarkName.text = track.name
            else:
                placemarkName.text = 'TBC'
            placemarkStyleUrl = etree.SubElement(placemark, 'styleUrl')
            placemarkStyleUrl.text = '#wizard'

            # LineString
            lineString = etree.SubElement(placemark, 'LineString')
            lineStringExtrude = etree.SubElement(lineString, 'extrude')
            lineStringExtrude.text = '1'       
            lineStringTessellate = etree.SubElement(lineString, 'tessellate')
            lineStringTessellate.text = '1'

            # Coordinates
            coordinatesList = []
            for i in range(track.numPoints):
                coordinate = coordinateFormat.format(track.data['lon'][i], track.data['lat'][i])
                coordinatesList.append(coordinate)

            coordinates = etree.SubElement(lineString, 'coordinates')
            coordinates.text=' '.join(coordinatesList)

        self.buffer = etree.tostring(kml, pretty_print=True, xml_declaration=False, encoding='UTF-8',
                                     doctype='<?xml version="1.0" encoding="UTF-8"?>')


    def save(self):
        '''Save KML to disk'''
        
        with open(self.filename, 'wb') as f:
            f.write(self.buffer)

## Unit Tests

In [3]:
class TestKmlWriter(unittest.TestCase):
    '''Class to test the KML writer'''

    def testKmlWriter(self):
        '''Test the KML writer'''

        filename = os.path.join(projdir, 'sessions', 'unittest.kml')

        kmlWriter = KmlWriter(filename, gpxReader.tracks)

        kmlWriter.save()
        
        os.unlink(filename)

In [4]:
if __name__ == '__main__':
    for path in ['python', '.', '..']:
        readersPath = os.path.join(path, 'core')
        if readersPath not in sys.path:
            sys.path.extend([readersPath])

    from file_reader import getFileReader

    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    gpxFilename = os.path.join(projdir, 'sessions', 'misc', 'Seatownandgoldencap.gpx')
    gpxReader = getFileReader(gpxFilename)

    pc1 = time.perf_counter()
    gpxReader.load()
    pc2 = time.perf_counter()

    print("\nTest file loaded in %0.2f seconds" % (pc2 - pc1))


Test file loaded in 0.01 seconds


In [5]:
if __name__ == '__main__':
    # Determine whether session is interactive or batch to facilitate unittest.main(..., exit=testExit)
    import __main__ as main
    testExit = hasattr(main, '__file__')

    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

.
----------------------------------------------------------------------
Ran 1 test in 0.004s

OK
